# Practico 5 (parte 5)

## Visualizar los word embeddings que entrenamos

* [Importación de módulos y librerías](#Importación-de-módulos-y-librerías)
* [Lectura del archivo de mensajes](#Lectura-del-archivo-de-mensajes)
* [Curación del dataset](#Curación-del-dataset)
* [Entrenamiento de los diferentes embeddings (Word2Vec)](#Entrenamiento-de-los-diferentes-embeddings-(Word2Vec))
* [Resumen de información de los word embedding generados](#Resumen-de-información-de-los-word-embedding-generados)
* [Computo de similitud](#Computo-de-similitud)
* [Referencias](#Referencias)

## Importación de módulos y librerías

In [ ]:
# Inclusion de librerias y módulos
import os
import logging
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Algunas utilidades
from utiles import bcolors

# Importamos wrod2vec de la lib gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# Importamos PCA de sklear
from sklearn.decomposition import PCA

# Importamos logger para tener informacion de estado
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Fijamos el estilo a seaborn
plt.style.use('seaborn')

# Fijamos la semilla del random
np.random.seed(42)

# Establecimos el muestreo del vocabulario
Nsamples = 200

## Definición de que vamos a mostrar del embedding y como vamos a hacerlo

Por cada embeding que tenemos vamos a presentar:

* El nombre del archivo que lo contiene, el cual tiene codificado los hiperparametros como se presento en ejercicio anterior
> YYYYMMDD-hhmmss_model_{size}-{window}-{min_count}-{sg}.bin
* El tamaño del vocabulario que maneja 
* Las primeras 50 palabras que posee
* Las últimas 50 palabras que posee
* Un scatter plot de un sub grupo de `Nsamples` palabras del embedding, un conjunto de palabras con connotación positiva y un conjunto de emojis con connotación positiva.

In [ ]:
def plot_embeding(modelfile, Nsamples=200, positives=None, emojis=None, useglove=False):
    #############################
    # Cargamos el embedding de preferencia, incluyendo la logica para Glove
    if useglove:
        model =  KeyedVectors.load_word2vec_format(modelfile, binary=False)
    else:
        model = Word2Vec.load(modelfile)
    
    #############################
    # Imprimimos el caso nombre del archivo del embedding, el tamaño del vocabulario
    # las primeras 50 palabras del vocabulario y la últimas 50 palabras del vocabulario
    print(f'{bcolors.HEADER}{modelfile} - Tamaño del vocabulario {len(list(model.wv.vocab))}{bcolors.ENDC}')
    print(f'{bcolors.OKGREEN}Primeras 50 compoentes del vocabulario{bcolors.ENDC}')
    print(f'{bcolors.OKBLUE}{list(model.wv.vocab.keys())[0:50]}{bcolors.ENDC}')
    print(f'{bcolors.OKGREEN}Ultimas 50 compoentes del vocabulario{bcolors.ENDC}')
    print(f'{bcolors.OKBLUE}{list(model.wv.vocab.keys())[-50:]}{bcolors.ENDC}')
    print(f'{bcolors.FAIL}#######################################{bcolors.ENDC}')


    #############################
    # Se genera el embedding de cada palabra del vocabulario
    X = model[model.wv.vocab]
    
    # Se genera el embedding de las palabras con connotación positiva
    if positives: Xp = model[positives]
    
    # Se genera el embedding de los emojis con connotación positiva
    if emojis: Xe = model[emojis]

    # Se hace reducción de dimensionalidad por medio de PCA
    # sobre los embeddings generados
    pca = PCA(n_components=2)
    result = pca.fit_transform(X)
    if positives: resultp = pca.transform(Xp)
    if emojis: resulte = pca.transform(Xe)


    #############################
    # create a scatter plot of the projection
    index = np.random.choice(range(0, result.shape[0]), Nsamples, replace=False)
    
    # Plot de los embeddings luego de aplicar PCA
    plt.figure(figsize=(20,16))
    p1 = sns.scatterplot(x=result[index, 0], y=result[index, 1], s=40, label='Vocabulario')
    if positives: p2 = sns.scatterplot(x=resultp[:, 0], y=resultp[:, 1], s=80, color='red', label='Palabras positivas')
    if emojis: p3 = sns.scatterplot(x=resulte[:, 0], y=resulte[:, 1], s=80, color='green', label='Emojis positivos')

    words = np.array(list(model.wv.vocab))[index]
    result = result[index,:]
    for i,word in enumerate(words):
        p1.text(result[i,0]+0.1, result[i,1], word,horizontalalignment='left', size='medium', color='black')#, weight='semibold')
    
    if positives:
        for i,word in enumerate(positives):
            p2.text(resultp[i,0]+0.1, resultp[i,1], word,horizontalalignment='left', size='large', color='red')#, weight='semibold')
    
    if emojis:
        for i,word in enumerate(emojis):
            p3.text(resulte[i,0]+0.1, resulte[i,1], word,horizontalalignment='left', size='large', color='green')#, weight='semibold')

    plt.xlabel('X', fontsize=14)
    plt.ylabel('Y', fontsize=14)
    plt.title('Word2Vec + PCA', fontsize=14)
    plt.legend(fontsize=14)
    sns.despine()

In [ ]:
positives = ['great', 'excellent', 'nice', 'awesome', 'amazing', 'fantastic', 'wonderful', 'perfect', 'nailed', 'brilliant', 'lovely', 'superb', 'beautiful','yupppppp']
emojis = [':grinning_face_with_smiling_eyes:', ':smiling_face_with_smiling_eyes:', ':face_with_tears_of_joy:', ':grinning_face:', ':smiling_face_with_sunglasses:',':smiling_face_with_open_mouth_&_smiling_eyes:']

In [ ]:
modelfile = os.path.join('.','20201008-140336_model_100-5-1-0.bin')
plot_embeding(modelfile, Nsamples=Nsamples, positives=positives, emojis=emojis)

In [ ]:
modelfile = os.path.join('.','20201008-140436_model_100-5-100-0.bin')
plot_embeding(modelfile, Nsamples=Nsamples, positives=positives, emojis=emojis)

In [ ]:
positives = ['great', 'excellent', 'nice', 'awesome', 'amazing', 'fantastic', 'wonderful', 'perfect', 'brilliant', 'superb','yupppppp'] 
#, 'nailed', 'lovely', 'beautiful'

emojis = [':smiling_face_with_smiling_eyes:', ':face_with_tears_of_joy:', ':smiling_face_with_sunglasses:',':smiling_face_with_open_mouth_&_smiling_eyes:']
#':grinning_face_with_smiling_eyes:',  ':grinning_face:',

modelfile = os.path.join('.','20201008-140513_model_100-5-300-0.bin')
plot_embeding(modelfile, Nsamples=Nsamples, positives=positives, emojis=emojis)

In [ ]:
modelfile = os.path.join('.','20201008-142337_model_100-1-300-0.bin')
plot_embeding(modelfile, Nsamples=Nsamples, positives=positives, emojis=emojis)

-------------------

In [ ]:
# glove_input_file = '../glove.6B/glove.6B.100d.txt'
# word2vec_output_file = '../glove.6B/glove.6B.100d.txt.word2vec'
# glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
positives = ['thanks','great', 'excellent', 'nice', 'awesome', 'amazing', 'fantastic', 'wonderful', 'perfect', 'nailed', 'brilliant', 'lovely', 'superb', 'beautiful'] #, 'yupppppp'
# emojis = [':grinning_face_with_smiling_eyes:', ':smiling_face_with_smiling_eyes:', ':face_with_tears_of_joy:', ':grinning_face:', ':smiling_face_with_sunglasses:',':smiling_face_with_open_mouth_&_smiling_eyes:']

modelfile = os.path.join('..', 'glove.6B', 'glove.6B.100d.txt.word2vec')
plot_embeding(modelfile, Nsamples=Nsamples, positives=positives, emojis=None, useglove=True)